In [1]:
import pandas as pd 
import numpy as np

from plotnine import *
from itertools import product
from scipy.stats import norm
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Lasso

C:\Users\rober\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\rober\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (
C:\Users\rober\AppData\Local\Temp/ipykernel_33556/1374421021.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def black_scholes_price(S, K, r, T, sigma):
    """Calculate Black Scholes option price."""
  
    d1 = (np.log(S/K)+(r+sigma**2/2)*T)/(sigma*np.sqrt(T))
    d2 = d1-sigma*np.sqrt(T)
    price = S*norm.cdf(d1)-K*np.exp(-r*T)*norm.cdf(d2)
    
    return price

Where ML comes handy: We illustrate the concept of ML by showing how ML methods learn the Black-Scholes equation after observing some different specifications and corresponding prices without us revealing the exact pricing equation.

 - Start with simulated data
 - Compute option prices for call options for a **grid** of different combinations of times to maturity (T), risk-free rates, volatilies (sigma), strike prices (K), and current stock prices (S). 
 -  In the code below, we add an idiosyncratic error term to each observation such that the prices considered do not exactly reflect the values implied by the Black-Scholes equation.

In [6]:
random_state = 42
np.random.seed(random_state) #to keep the analysis reproducible 

S = np.arange(40, 61) #numpy.arange returns evenly spaced values within a given interval. When not spec the interval is 1
K = np.arange(20, 91) #inteval is specified as the third argument
r = np.arange(0, 0.051, 0.01)
T = np.arange(3/12, 2.01, 1/12)
sigma = np.arange(0.1, 0.81, 0.1)

option_prices = pd.DataFrame(
  product(S, K, r, T, sigma), #is product a binomial coeff of combinations? #in dataframe
  columns=["S", "K", "r", "T", "sigma"]
)

option_prices["black_scholes"] = black_scholes_price(
  option_prices["S"].values, 
  option_prices["K"].values, 
  option_prices["r"].values, 
  option_prices["T"].values, 
  option_prices["sigma"].values
)

option_prices = (option_prices
  .assign(
    observed_price=lambda x: (
      x["black_scholes"] + np.random.normal(scale=0.15) #random error
    )
  )
)

In [8]:
option_prices

,S,K,r,T,sigma,black_scholes,observed_price
0,40,20,0.00,0.25,0.1,20.000000,20.074507
1,40,20,0.00,0.25,0.2,20.000000,20.074507
2,40,20,0.00,0.25,0.3,20.000002,20.074509
3,40,20,0.00,0.25,0.4,20.000377,20.074884
4,40,20,0.00,0.25,0.5,20.005859,20.080367
...,...,...,...,...,...,...,...
1574491,60,90,0.05,2.00,0.4,7.193144,7.267651
1574492,60,90,0.05,2.00,0.5,10.528885,10.603392
1574493,60,90,0.05,2.00,0.6,13.911119,13.985626
1574494,60,90,0.05,2.00,0.7,17.267163,17.341670


In [7]:
#train_test_split

train_data, test_data = train_test_split(
  option_prices, 
  test_size=0.01, random_state=random_state
)

In [13]:
#preprocess the data
preprocessor = ColumnTransformer(
  transformers=[(
    "normalize_predictors", 
     StandardScaler(),
     ["S", "K", "r", "T", "sigma"]
  )],
  remainder="drop"
)

## Single layer networks and random forests

How to fit a neural network to the data. The function MLPRegressor() from the package scikit-learn provides the functionality to initialize a single layer, feed-forward neural network. The specification below defines a single layer feed-forward neural network with ten hidden units. We set the number of training iterations to max_iter=1000.

In [14]:
max_iter = 1000

nnet_model = MLPRegressor(
  hidden_layer_sizes=10, 
  max_iter=max_iter, 
  random_state=random_state
)

We can follow the straightforward workflow as in the chapter before: define a workflow, equip it with the recipe, and specify the associated model. Finally, fit the model with the training data.

In [15]:
nnet_pipeline = Pipeline([
  ("preprocessor", preprocessor),
  ("regressor", nnet_model)
])

nnet_fit = nnet_pipeline.fit(
  train_data.drop(columns=["observed_price"]), 
  train_data.get("observed_price")
)

In [16]:
#random forest

rf_model = RandomForestRegressor(
  n_estimators=50, 
  min_samples_leaf=2000, 
  random_state=random_state
)

rf_pipeline = Pipeline([
  ("preprocessor", preprocessor),
  ("regressor", rf_model)
])

rf_fit = rf_pipeline.fit(
  train_data.drop(columns=["observed_price"]), 
  train_data.get("observed_price")
)

## Deep neural networks

A deep neural network is a neural network with multiple layers between the input and output layers. By chaining multiple layers together, more complex structures can be represented with fewer parameters than simple shallow (one-layer) networks as the one implemented above. 
The following code chunk implemenets a deep neural network with three hidden layers of size ten each and logistic activation functions.

In [17]:
deepnnet_model = MLPRegressor(
  hidden_layer_sizes=(10, 10, 10),
  activation="logistic", 
  solver="lbfgs",
  max_iter=max_iter, 
  random_state=random_state
)
                              
deepnnet_pipeline = Pipeline([
  ("preprocessor", preprocessor),
  ("regressor", deepnnet_model)
])

deepnnet_fit = deepnnet_pipeline.fit(
  train_data.drop(columns=["observed_price"]),
  train_data.get("observed_price")
)

C:\Users\rober\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html


## Universal approximation

Before we evaluate the results, we implement one more model. In principle, any non-linear function can also be approximated by a linear model containing the input variables’ polynomial expansions.
To illustrate this, we include polynomials up to the fifth degree of each predictor and then add all possible pairwise interaction terms.

In [ ]:
lm_pipeline = Pipeline([
  ("polynomial", PolynomialFeatures(degree=5, 
                                    interaction_only=False, 
                                    include_bias=True)),
  ("scaler", StandardScaler()),
  ("regressor", Lasso(alpha=0.01))
])

lm_fit = lm_pipeline.fit(
  train_data.get(["S", "K", "r", "T", "sigma"]),
  train_data.get("observed_price")
)

Collect all predictions to compare the out-of-sample prediction error evaluated on ten thousand new data points.

In [ ]:
test_X = test_data.get(["S", "K", "r", "T", "sigma"])
test_y = test_data.get("observed_price")

predictive_performance = (pd.concat(
    [test_data.reset_index(drop=True), 
     pd.DataFrame({"Random forest": rf_fit.predict(test_X),
                   "Single layer": nnet_fit.predict(test_X),
                   "Deep NN": deepnnet_fit.predict(test_X),
                   "Lasso": lm_fit.predict(test_X)})
    ], axis=1)
  .melt(
    id_vars=["S", "K", "r", "T", "sigma",
             "black_scholes", "observed_price"],
    var_name="Model",
    value_name="Predicted"
  )
  .assign(
    moneyness=lambda x: x["S"]-x["K"],
    pricing_error=lambda x: np.abs(x["Predicted"]-x["black_scholes"])
  )
)

In [ ]:
predictive_performance_plot = (
  ggplot(predictive_performance, 
         aes(x="moneyness", y="pricing_error")) +
  geom_point(alpha=0.05) +
  facet_wrap("Model") + 
  labs(x="Moneyness (S - K)", y="Absolut prediction error (USD)",
       title="Prediction errors of call options for different models") +
  theme(legend_position="")
)
predictive_performance_plot.draw()